***
# Adidas Shoe Data Extraction and Wrangling
***

In [1]:
import pandas as pd
import re
import spacy
import pickle
pd.set_option("display.max_columns", None)
import os
from selenium import webdriver
from  selenium.webdriver.common.by import By
from  selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import requests
import json

os.environ["PATH"] += r"C:/SeleniumDrivers"

In [247]:
def find_shoe_review(shoe_number):
    
    url = "https://www.adidas.com/us"
    driver = webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()
    driver.implicitly_wait(15)

    try:
        accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
        accept_button.click()
    except:
        print("Accept button not found!")

    search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
    search.click()
    search.send_keys("shoe")
    search.send_keys(Keys.ENTER)

    main_content = driver.find_element(By.ID, "main-content")
    products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
    products[shoe_number].click()

    try:
        pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
        pop_up_button.click()
    except NoSuchElementException:
        ("print no pop up")

    try:
        driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
    #<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
    except:
        print("Not Found")

    try:
        product_name = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
    except NoSuchElementException:
        product_name = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

    try:
        product_price = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
    except NoSuchElementException:
        product_price = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item notranslate"]').get_attribute("innerHTML")
    #<div class="gl-price-item notranslate">$100</div>  
    try:
        product_colour = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
    except NoSuchElementException:
        product_colour = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
    #<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
    print(product_name)
    print(product_price)
    print(product_colour)

    try:
        review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
        review_drop_down.click()
    except NoSuchElementException:
        review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
        review_drop_down.click()
    except NoSuchElementException:
        review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
        review_drop_down.click()
    for i in range (100):
        load_more_reviews = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="ratings-load-more"]')
        load_more_reviews.click()


    try:
        read_mores = driver.find_elements(By.CSS_SELECTOR, '[data-auto-id="review-read-more-button"]')
        #<button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>
    except:
         read_mores = driver.find_elements(By.CSS_SELECTOR, '[class="read-more___2dacS gl-link gl-label--m"]')


    for reading in read_mores[:400]:
        reading.click()

    heading = []
    messaging = []
    info = []

    #actual reviews
    reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
    #<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
    for review in reviews:
        header = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>

        try:
            message = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
            #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
        except:
            try:
                message = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
            except:
                message = None

        try:
            review_info = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
            #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
        except:
            review_info = None

        heading.append(header)
        messaging.append(message)
        info.append(review_info)
        
        driver.close()
        
    return heading, messaging, info

In [81]:
#product 1
#samba OG shoes

url = "https://www.adidas.com/us"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(15)

try:
    accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
    accept_button.click()
except:
    print("Accept button not found!")

search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
search.click()
search.send_keys("shoe")
search.send_keys(Keys.ENTER)

main_content = driver.find_element(By.ID, "main-content")
products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
products[1].click()

try:
    pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
    pop_up_button.click()
except NoSuchElementException:
    ("print no pop up")

try:
    driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
#<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
except:
    print("Not Found")
    
try:
    product_name = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_name = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

try:
    product_price = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_price = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item notranslate"]').get_attribute("innerHTML")
#<div class="gl-price-item notranslate">$100</div>  
try:
    product_colour = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
except NoSuchElementException:
    product_colour = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
#<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
print(product_name)
print(product_price)
print(product_colour)

try:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
    review_drop_down.click()
for i in range (100):
    load_more_reviews = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="ratings-load-more"]')
    load_more_reviews.click()

    
try:
    read_mores = driver.find_elements(By.CSS_SELECTOR, '[data-auto-id="review-read-more-button"]')
    #<button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>
except:
     read_mores = driver.find_elements(By.CSS_SELECTOR, '[class="read-more___2dacS gl-link gl-label--m"]')


for reading in read_mores[:400]:
    reading.click()
    
heading = []
messaging = []
info = []

#actual reviews
reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
#<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
for review in reviews:
    header = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
    #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>
    
    try:
        message = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
        #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
    except:
        try:
            message = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        except:
            message = None
    
    try:
        review_info = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
        #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
    except:
        review_info = None
        
    heading.append(header)
    messaging.append(message)
    info.append(review_info)

Not Found
<span>Samba OG Shoes</span>
$100
6 colors available


In [89]:
len(heading), len(messaging), len(info)

(242, 242, 242)

In [285]:
heading

['<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Nice quality shoes</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Nice shoes</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Buy the shoes!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">ALL GOODS SOLID! WORTH THE PRICE!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Never out of style</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">The perfect everyday shoe</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Super nice shoes</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">ALL GOODS SOLID! WORTH THE PRICE!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Never out of style</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">The perfect everyday shoe</strong>',
 '<strong class="tit

In [439]:
heading1_clean = []

for i,head in enumerate(heading):
    clean_head = heading[i][63:].replace("</strong>", "")
    heading1_clean.append(clean_head)
    
len(heading1_clean)

242

In [287]:
heading1_clean

['Nice quality shoes',
 'Nice shoes',
 'Buy the shoes!',
 'ALL GOODS SOLID! WORTH THE PRICE!',
 'Never out of style',
 'The perfect everyday shoe',
 'Super nice shoes',
 'ALL GOODS SOLID! WORTH THE PRICE!',
 'Never out of style',
 'The perfect everyday shoe',
 'Super nice shoes',
 'Nice kicks,size up!',
 'ALL GOODS SOLID! WORTH THE PRICE!',
 'Never out of style',
 'The perfect everyday shoe',
 'Super nice shoes',
 'Nice kicks,size up!',
 'The perfect everyday shoe',
 'Super nice shoes',
 'Nice kicks,size up!',
 'I wore these in Italy for 16,000 steps/day. Great!',
 'So so',
 'So so',
 'Classic and will never ran out of style',
 'Great look',
 'Great shoes!',
 'Best purchase ever',
 'Perfectly comfortable and on-trend footwear! Love!',
 'Satisfied customer',
 'Super buy',
 'Great buy, very wearable style',
 'Samba OG Shoes- the best everyday shoes!',
 'Perfectly comfortable and on-trend footwear! Love!',
 'Satisfied customer',
 'Super buy',
 'Great buy, very wearable style',
 'Samba OG 

In [84]:
messaging

['<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">They match practically with  any outfit that I wear!</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Very nice shoes , just not as green as the picture.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Bang on trend, comfy and cool. Would recommend these to anyone</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">THAT WAS DOPE! medyo mahaba lng ng very little lang naman kasi 9.5 UK pala is size 10 US. i was expecting na size 9.5 UK at the same time 9.5 US too . but its all good no worries. SAMBA OG IS LIT!</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Love how light on your feet they are and comfortable. I’ve been a Samba wearer for years and they never get old.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-n

In [440]:
messaging1_clean = []

for i,message in enumerate(messaging):
    clean_message = messaging[i][84:].replace("</div>", "")
    messaging1_clean.append(clean_message)
    
len(messaging1_clean)

242

In [305]:
messaging1_clean

['They match practically with  any outfit that I wear!',
 'Very nice shoes , just not as green as the picture.',
 'Bang on trend, comfy and cool. Would recommend these to anyone',
 'THAT WAS DOPE! medyo mahaba lng ng very little lang naman kasi 9.5 UK pala is size 10 US. i was expecting na size 9.5 UK at the same time 9.5 US too . but its all good no worries. SAMBA OG IS LIT!',
 'Love how light on your feet they are and comfortable. I’ve been a Samba wearer for years and they never get old.',
 'You can wear these shoes with anything and they fit comfortably',
 'Love the colorway and the way it fits, what an amazing shoe!',
 'THAT WAS DOPE! medyo mahaba lng ng very little lang naman kasi 9.5 UK pala is size 10 US. i was expecting na size 9.5 UK at the same time 9.5 US too . but its all good no worries. SAMBA OG IS LIT!',
 'Love how light on your feet they are and comfortable. I’ve been a Samba wearer for years and they never get old.',
 'You can wear these shoes with anything and they f

In [311]:
messaging1_clean[25]

'Great shoes!  Feels great and looks great! I would recommend these to anyone.  My wife is waiting on a pair in her size and color now-'

In [85]:
info

['abdubs35 | September 13, 2023',
 'THEMAN | September 13, 2023 | Incentivized Review',
 'Huggsy | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'TOYOTABOY | September 12, 2023',
 'SangeBo | September 12, 2023',
 'Herryhernadez | September 12, 2023',
 'Vides | September 12, 2023',
 'TOYOTABOY | September 12, 2023',
 'SangeBo | September 12, 2023',
 'Herryhernadez | September 12, 2023',
 'Vides | September 12, 2023',
 'Ali7 | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'TOYOTABOY | September 12, 2023',
 'SangeBo | September 12, 2023',
 'Herryhernadez | September 12, 2023',
 'Vides | September 12, 2023',
 'Ali7 | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'Herryhernadez | September 12, 2023',
 'Vides | September 12, 2023',
 'Ali7 | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'Julie0515 | September 12, 2023',
 'Bear | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'Bear | September 12, 202

In [316]:
product_name[6:].replace("</span>", "")

'Samba OG Shoes'

In [441]:
len(messaging1_clean)

242

In [443]:
shoe1 = pd.DataFrame(
    {"Title": [product_name[6:].replace("</span>", "")]*len(messaging1_clean),
    "Price":[product_price[1:]]*len(messaging1_clean),
    "ColoursAvailable": [product_colour[0]]*len(messaging1_clean),
    "ReviewTopic": heading1_clean,
    "Reviews": messaging1_clean,
    "ReviewInfo": info}
)
shoe1

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,Samba OG Shoes,100,6,Nice quality shoes,They match practically with any outfit that I...,"abdubs35 | September 13, 2023"
1,Samba OG Shoes,100,6,Nice shoes,"Very nice shoes , just not as green as the pic...","THEMAN | September 13, 2023 | Incentivized Review"
2,Samba OG Shoes,100,6,Buy the shoes!,"Bang on trend, comfy and cool. Would recommend...","Huggsy | September 12, 2023 | Verified Purchas..."
3,Samba OG Shoes,100,6,ALL GOODS SOLID! WORTH THE PRICE!,THAT WAS DOPE! medyo mahaba lng ng very little...,"TOYOTABOY | September 12, 2023"
4,Samba OG Shoes,100,6,Never out of style,Love how light on your feet they are and comfo...,"SangeBo | September 12, 2023"
...,...,...,...,...,...,...
237,Samba OG Shoes,100,6,Great and fun,very comfy and i really enjoyed wearing them t...,"chinochubs | September 6, 2023"
238,Samba OG Shoes,100,6,hurts after walking for awhile,#aesthetic\n#kawaii\n#sopinterest\n#y2kqueen e...,"cunty | September 6, 2023"
239,Samba OG Shoes,100,6,Great stylish and comfortable shoe.,Nice grip and great for all activities. Could ...,"Mal14 | September 6, 2023"
240,Samba OG Shoes,100,6,"Amazing, and a calm shoe.",i love the colors and how it goes with differe...,"dswish | September 6, 2023"


In [92]:
#product 2
#samba OG shoes

url = "https://www.adidas.com/us"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(15)

try:
    accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
    accept_button.click()
except:
    print("Accept button not found!")

search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
search.click()
search.send_keys("shoe")
search.send_keys(Keys.ENTER)

main_content = driver.find_element(By.ID, "main-content")
products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
products[8].click()

try:
    pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
    pop_up_button.click()
except NoSuchElementException:
    ("print no pop up")

try:
    driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
#<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
except:
    print("Not Found")
    
try:
    product_name2 = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_name = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

try:
    product_price2 = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_price2 = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item notranslate"]').get_attribute("innerHTML")
#<div class="gl-price-item notranslate">$100</div>  
try:
    product_colour2 = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
except NoSuchElementException:
    product_colour2 = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
#<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
print(product_name2)
print(product_price2)
print(product_colour2)

try:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
    review_drop_down.click()
for i in range (100):
    load_more_reviews = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="ratings-load-more"]')
    load_more_reviews.click()

    
try:
    read_mores = driver.find_elements(By.CSS_SELECTOR, '[data-auto-id="review-read-more-button"]')
    #<button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>
except:
     read_mores = driver.find_elements(By.CSS_SELECTOR, '[class="read-more___2dacS gl-link gl-label--m"]')


for reading in read_mores[:400]:
    reading.click()
    
heading2 = []
messaging2 = []
info2 = []

#actual reviews
reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
#<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
for review in reviews:
    header2 = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
    #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>
    
    try:
        message2 = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
        #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
    except:
        try:
            message2 = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        except:
            message2 = None
    
    try:
        review_info2 = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
        #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
    except:
        review_info2 = None
        
    heading2.append(header2)
    messaging2.append(message2)
    info2.append(review_info2)

Not Found
<span>Samba OG Shoes</span>
$100
6 colors available


In [93]:
len(heading2), len(messaging2), len(info2)

(287, 287, 287)

In [94]:
heading2

['<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Happy Purchase</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">love it</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">good</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Really comfy, little small but still ok sizewise</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Love.</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Timeless design and of great quality!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Good quality, comfortable and stylish</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Seems cheaply made for 100 go with everything</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">A need.</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Best</strong>',
 '<strong class="title

In [343]:
heading2[283][46:]

'Great quality &amp; comfy!</strong>'

In [344]:
heading2_clean = []

for i,head in enumerate(heading2):
    if '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">' in head:
        clean_head = heading2[i][63:].replace("</strong>", "")
        heading2_clean.append(clean_head)
    else:
        clean_head = heading2[i][46:].replace("</strong>", "")
        heading2_clean.append(clean_head)
    
len(heading2_clean)

287

In [345]:
heading2_clean

['Happy Purchase',
 'love it',
 'good',
 'Really comfy, little small but still ok sizewise',
 'Love.',
 'Timeless design and of great quality!',
 'Good quality, comfortable and stylish',
 'Seems cheaply made for 100 go with everything',
 'A need.',
 'Best',
 'bomb',
 'loveeee the shoess',
 'Would purchase another',
 'Run large for women but amazing with the right siz',
 'Seems cheaply made for 100 go with everything',
 'A need.',
 'Best',
 'loveeee the shoess',
 'Happy Purchase',
 'love it',
 'i love it',
 'good',
 'So so',
 'Classic and will never ran out of style',
 'Great look',
 'Great shoes!',
 'Best purchase ever',
 'loveeee the shoess',
 'Happy Purchase',
 'love it',
 'i love it',
 'good',
 'Nice quality shoes',
 'Overall good shoe',
 'Nice shoes',
 'Buy the shoes!',
 'Never out of style',
 'Nice quality shoes',
 'Overall good shoe',
 'Nice shoes',
 'Buy the shoes!',
 'Never out of style',
 'I wore these in Italy for 16,000 steps/day. Great!',
 'So so',
 'Classic and will never 

In [95]:
messaging2

['<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">I love my new shoes! They are comfortable and are a great "dress" sneaker"</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">its very comfortable, fits perfectly, size is perfect</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">have bought few things which are good so go buy them</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">love the runners, so comfy and really like the style</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Love them. I sized up from a 5 to 5.5, and they\'re perfect.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Sambas are sleek and can go with any of your casual outfits!\nIf you’re a daily beater that is fashionable and can of great quality then don’t look 

In [373]:
messaging2[286][67:].replace('</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>',"")

'I used to have Lucas push skate (similar to sambas) shoes since 2018 and they finally wore out and these were on par with the exception of breathability although I’m still breaking them in. I love the real leather though as well as the colors. Great sambas as usual. I’ve been wearing them since 2006 pretty much non stop and they’re some of the best shoes ever made. Will be purchasing a new pair in a few years once these get a hole in them.'

In [374]:
messaging2_clean = []

for i,message in enumerate(messaging2):
    if '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">' in message:
        clean_message = messaging2[i][84:].replace("</div>", "")
        messaging2_clean.append(clean_message)
    else:
        clean_message = messaging2[i][67:].replace('</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>', "")
        messaging2_clean.append(clean_message)
    
len(messaging2_clean)

287

In [375]:
messaging2_clean

['I love my new shoes! They are comfortable and are a great "dress" sneaker"',
 'its very comfortable, fits perfectly, size is perfect',
 'have bought few things which are good so go buy them',
 'love the runners, so comfy and really like the style',
 "Love them. I sized up from a 5 to 5.5, and they're perfect.",
 'Sambas are sleek and can go with any of your casual outfits!\nIf you’re a daily beater that is fashionable and can of great quality then don’t look too far.\nAbout comfort though one must realise this isn’t a cushioned shoe, it was designed originally for indoor football so the cushion is minimal.\nIt is a flat shoe which I love as I don’t prefer to wear cushioned shoes apart from for running.\n\nIf you have wide feet like me, then go 1/2 size bigger for a wider fit as the shoes runs narrow.\n\nI’m so stoked for buying this!',
 'Good quality, comfortable &amp; stylish.\n\nGood quality, comfortable &amp; stylish.',
 'They crease super easily but I get a lot of compliments on 

In [96]:
info2

['DMM35 | September 13, 2023 | Verified Purchaser | Incentivized Review',
 'salome | September 13, 2023',
 'bai22 | September 13, 2023',
 'ann7 | September 13, 2023 | Verified Purchaser | Incentivized Review',
 'SophieH91 | September 13, 2023 | Verified Purchaser | Incentivized Review',
 'Bharath | September 13, 2023',
 'ADEE | September 13, 2023',
 'Csarge | September 14, 2023',
 'Liz2 | September 13, 2023 | Verified Purchaser | Incentivized Review',
 'Jbhd | September 13, 2023',
 'anna555 | September 13, 2023',
 'eya25 | September 13, 2023',
 'Les65 | September 14, 2023 | Incentivized Review',
 'SN76 | September 14, 2023 | Verified Purchaser | Incentivized Review',
 'Csarge | September 14, 2023',
 'Liz2 | September 13, 2023 | Verified Purchaser | Incentivized Review',
 'Jbhd | September 13, 2023',
 'eya25 | September 13, 2023',
 'DMM35 | September 13, 2023 | Verified Purchaser | Incentivized Review',
 'salome | September 13, 2023',
 'adrianaflorixnx | September 13, 2023 | Verified Pu

In [445]:
shoe2 = pd.DataFrame(
    {"Title": [product_name2[6:].replace("</span>", "")]*len(messaging2),
     "Price":[product_price2[1:]]*len(messaging2),
     "ColoursAvailable": [product_colour[0]]*len(messaging2_clean),
    "ReviewTopic": heading2_clean,
    "Reviews": messaging2_clean,
    "ReviewInfo": info2}
)
shoe2

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,Samba OG Shoes,100,6,Happy Purchase,I love my new shoes! They are comfortable and ...,"DMM35 | September 13, 2023 | Verified Purchase..."
1,Samba OG Shoes,100,6,love it,"its very comfortable, fits perfectly, size is ...","salome | September 13, 2023"
2,Samba OG Shoes,100,6,good,have bought few things which are good so go bu...,"bai22 | September 13, 2023"
3,Samba OG Shoes,100,6,"Really comfy, little small but still ok sizewise","love the runners, so comfy and really like the...","ann7 | September 13, 2023 | Verified Purchaser..."
4,Samba OG Shoes,100,6,Love.,"Love them. I sized up from a 5 to 5.5, and the...","SophieH91 | September 13, 2023 | Verified Purc..."
...,...,...,...,...,...,...
282,Samba OG Shoes,100,6,Best shoe ever!,I think this product stands out because it’s n...,None
283,Samba OG Shoes,100,6,Great quality &amp; comfy!,Finally got these after a year! Lol. They're ...,None
284,Samba OG Shoes,100,6,A true classic,My first pair was in 85 and I’m just as happy ...,None
285,Samba OG Shoes,100,6,stylish,"the shoes go with everything, they fit perfect...",None


In [98]:
#product 3
#Racer TR23

url = "https://www.adidas.com/us"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(15)

try:
    accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
    accept_button.click()
except:
    print("Accept button not found!")

search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
search.click()
search.send_keys("shoe")
search.send_keys(Keys.ENTER)

main_content = driver.find_element(By.ID, "main-content")
products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
products[15].click()

try:
    pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
    pop_up_button.click()
except NoSuchElementException:
    ("print no pop up")

try:
    driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
#<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
except:
    print("Not Found")
    
try:
    product_name3 = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_name3 = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

try:
    product_price3 = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_price3 = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item notranslate"]').get_attribute("innerHTML")
#<div class="gl-price-item notranslate">$100</div>  
try:
    product_colour3 = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
except NoSuchElementException:
    product_colour3 = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
#<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
print(product_name3)
print(product_price3)
print(product_colour3)

try:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
    review_drop_down.click()
for i in range (100):
    load_more_reviews = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="ratings-load-more"]')
    load_more_reviews.click()

    
try:
    read_mores = driver.find_elements(By.CSS_SELECTOR, '[data-auto-id="review-read-more-button"]')
    #<button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>
except:
     read_mores = driver.find_elements(By.CSS_SELECTOR, '[class="read-more___2dacS gl-link gl-label--m"]')


for reading in read_mores[:400]:
    reading.click()
    
heading3 = []
messaging3 = []
info3 = []

#actual reviews
reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
#<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
for review in reviews:
    header3 = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
    #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>
    
    try:
        message3 = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
        #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
    except:
        try:
            message3 = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        except:
            message3 = None
    
    try:
        review_info3 = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
        #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
    except:
        review_info3 = None
        
    heading3.append(header3)
    messaging3.append(message3)
    info3.append(review_info3)

Not Found
<span>Racer TR23 Shoes</span>
$80
10 colors available


In [126]:
len(heading3), len(messaging3),len(info3)

(96, 96, 96)

In [100]:
heading3

['<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Very comfortable and smart.</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Stylish, Comfortable and light</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Worth it</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Very good product</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Purchased for a friend and he absolutely LOVES!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Great purchase.</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">I ordered size 12.5 and they feel like size 11.</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Very good product</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Purchased for a friend and he absolutely LOVES!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expan

In [379]:
heading3[5][63:]

'Great purchase.</strong>'

In [380]:
heading3_clean = []

for i,head in enumerate(heading3):
    clean_head = heading3[i][63:].replace("</strong>", "")
    heading3_clean.append(clean_head)
    
len(heading3_clean)

96

In [381]:
heading3_clean

['Very comfortable and smart.',
 'Stylish, Comfortable and light',
 'Worth it',
 'Very good product',
 'Purchased for a friend and he absolutely LOVES!',
 'Great purchase.',
 'I ordered size 12.5 and they feel like size 11.',
 'Very good product',
 'Purchased for a friend and he absolutely LOVES!',
 'Great purchase.',
 'I ordered size 12.5 and they feel like size 11.',
 'The specials are the best',
 'Very good product',
 'Purchased for a friend and he absolutely LOVES!',
 'Great purchase.',
 'I ordered size 12.5 and they feel like size 11.',
 'The specials are the best',
 'Very good product',
 'Purchased for a friend and he absolutely LOVES!',
 'Great purchase.',
 'I ordered size 12.5 and they feel like size 11.',
 'The specials are the best',
 'Very good product',
 'Purchased for a friend and he absolutely LOVES!',
 'Great purchase.',
 'I ordered size 12.5 and they feel like size 11.',
 'The specials are the best',
 'Nice comfy shoes',
 'Confortable for job',
 'Teen loves them!',
 'Gr

In [101]:
messaging3

['<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Very comfortable. Totally black so can get away with wearing with more formal clothes.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">You can wear it with anything and it’s very comfortable</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Worth it for the price you pay. It’s comfortable but could be more better</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Perfect size and looks good \nVery comfortable during walking</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Extra comfortable and not too flashy, great price as well!</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">I walk 10,000 steps daily and these sneakers feel great.</div>',
 '<div class="clamped___1_0yP gl-vspace g

In [392]:
messaging3[94][84:]

' me of the Guinness handball spezials from a couple of years back. Missed out on those but I might like these a little more.</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>'

In [400]:
messaging3_clean = []

for i,message in enumerate(messaging3):
    if i < 94:
        clean_message = messaging3[i][84:].replace("</div>", "")
        messaging3_clean.append(clean_message)
    else:
        clean_message = messaging3[i][84:].replace("</div>", "")
        clean_message = clean_message.replace('</div><div class="read-more-btn-container___2Q8_H"\
        ><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">\
        Read more</button></div>',"")
        jag = '<div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>'    
        if jag in clean_message:
            clean_message = clean_message.replace(jag, "")
            messaging3_clean.append(clean_message)
        else:
            messaging3_clean.append(clean_message)
    
len(messaging3_clean)

96

In [401]:
messaging3_clean

['Very comfortable. Totally black so can get away with wearing with more formal clothes.',
 'You can wear it with anything and it’s very comfortable',
 'Worth it for the price you pay. It’s comfortable but could be more better',
 'Perfect size and looks good \nVery comfortable during walking',
 'Extra comfortable and not too flashy, great price as well!',
 'I walk 10,000 steps daily and these sneakers feel great.',
 'This had to have been a pair that slipped through. Ordered same exact shoe in different color and fits perfect. These will be returned but the shoe model is awesome.',
 'Perfect size and looks good \nVery comfortable during walking',
 'Extra comfortable and not too flashy, great price as well!',
 'I walk 10,000 steps daily and these sneakers feel great.',
 'This had to have been a pair that slipped through. Ordered same exact shoe in different color and fits perfect. These will be returned but the shoe model is awesome.',
 "I think they should have Puerto Rico on the shipp

In [399]:
messaging3_clean[94:]

[' me of the Guinness handball spezials from a couple of years back. Missed out on those but I might like these a little more.<div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>',
 's on point. I was stopped multiple times wearing the sneakers!<div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>']

In [102]:
info3

['ScoP | September 13, 2023 | Verified Purchaser | Incentivized Review',
 'Kalabofc | September 13, 2023',
 'Krish | September 13, 2023 | Verified Purchaser | Incentivized Review',
 'Thala | September 13, 2023',
 'Sarah | September 13, 2023',
 'RTM26 | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'Remus | September 12, 2023',
 'Thala | September 13, 2023',
 'Sarah | September 13, 2023',
 'RTM26 | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'Remus | September 12, 2023',
 'Nicbrali | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'Thala | September 13, 2023',
 'Sarah | September 13, 2023',
 'RTM26 | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'Remus | September 12, 2023',
 'Nicbrali | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'Thala | September 13, 2023',
 'Sarah | September 13, 2023',
 'RTM26 | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'Remus | September 12, 2023

In [446]:
shoe3 = pd.DataFrame(
    {"Title": [product_name3[6:].replace("</span>", "")]*len(messaging3),
     "Price":[product_price3[1:]]*len(messaging3),
     "ColoursAvailable": [product_colour3[0]]*len(messaging3_clean),
    "ReviewTopic": heading3_clean,
    "Reviews": messaging3_clean,
    "ReviewInfo": info3}
)
shoe3

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,Campus 00s Shoes,110,7,Very comfortable and smart.,Very comfortable. Totally black so can get awa...,"ScoP | September 13, 2023 | Verified Purchaser..."
1,Campus 00s Shoes,110,7,"Stylish, Comfortable and light",You can wear it with anything and it’s very co...,"Kalabofc | September 13, 2023"
2,Campus 00s Shoes,110,7,Worth it,Worth it for the price you pay. It’s comfortab...,"Krish | September 13, 2023 | Verified Purchase..."
3,Campus 00s Shoes,110,7,Very good product,Perfect size and looks good \nVery comfortable...,"Thala | September 13, 2023"
4,Campus 00s Shoes,110,7,Purchased for a friend and he absolutely LOVES!,"Extra comfortable and not too flashy, great pr...","Sarah | September 13, 2023"
...,...,...,...,...,...,...
91,Campus 00s Shoes,110,7,Best all day walking shoe,Great for all day walking at the hospital and ...,"Vick90 | August 30, 2023"
92,Campus 00s Shoes,110,7,Love the gazelle,Colorway reminded me of the Guinness handball ...,"Izak | September 11, 2023"
93,Campus 00s Shoes,110,7,These sneakers never disappoint!,The color combo is on point. I was stopped mul...,"Sneakysneak | September 1, 2023"
94,Campus 00s Shoes,110,7,/strong>,me of the Guinness handball spezials from a c...,None


In [121]:
#product 4
#Gazelle Indoor Shoes

url = "https://www.adidas.com/us"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(15)

try:
    accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
    accept_button.click()
except:
    print("Accept button not found!")

search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
search.click()
search.send_keys("shoe")
search.send_keys(Keys.ENTER)

main_content = driver.find_element(By.ID, "main-content")
products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
products[12].click()

try:
    pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
    pop_up_button.click()
except NoSuchElementException:
    ("print no pop up")

try:
    driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
#<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
except:
    print("Not Found")
    
try:
    product_name4 = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_name4 = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

try:
    product_price4 = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_price4 = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item notranslate"]').get_attribute("innerHTML")
#<div class="gl-price-item notranslate">$100</div>  
try:
    product_colour4 = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
except NoSuchElementException:
    product_colour4 = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
#<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
print(product_name4)
print(product_price4)
print(product_colour4)

try:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
    review_drop_down.click()
    
heading4 = []
messaging4 = []
info4 = []

#actual reviews
reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
#<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
for review in reviews:
    header4 = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
    #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>
    
    try:
        message4 = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
        #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
    except:
        try:
            message4 = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        except:
            message4 = None
    
    try:
        review_info4 = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
        #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
    except:
        review_info4 = None
        
    heading3.append(header4)
    messaging3.append(message4)
    info3.append(review_info4)

Not Found
<span>Gazelle Indoor Shoes</span>
$120
Skip to add to bag


In [122]:
len(heading4), len(messaging4), len(info4)

(0, 0, 0)

In [448]:
shoe4 = pd.DataFrame(
    {"Title": [product_name4[6:].replace("</span>", "")],
     "Price":[product_price4[1:]],
     "ColoursAvailable": [1],
    "ReviewTopic": None,
    "Reviews": None,
    "ReviewInfo": None}
)
shoe4

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,Gazelle Indoor Shoes,120,1,None,None,None


In [125]:
#product 5
#Samba ADV Shoes

url = "https://www.adidas.com/us"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(15)

try:
    accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
    accept_button.click()
except:
    print("Accept button not found!")

search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
search.click()
search.send_keys("shoe")
search.send_keys(Keys.ENTER)

main_content = driver.find_element(By.ID, "main-content")
products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
products[21].click()

try:
    pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
    pop_up_button.click()
except NoSuchElementException:
    ("print no pop up")

try:
    driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
#<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
except:
    print("Not Found")
    
try:
    product_name5 = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_name5 = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

try:
    product_price5 = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_price5 = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item notranslate"]').get_attribute("innerHTML")
#<div class="gl-price-item notranslate">$100</div>  
try:
    product_colour5 = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
except NoSuchElementException:
    product_colour5 = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
#<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
print(product_name5)
print(product_price5)
print(product_colour5)

try:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
    review_drop_down.click()
    

heading5 = []
messaging5 = []
info5 = []

#actual reviews
reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
#<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
for review in reviews:
    header5 = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
    #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>
    
    try:
        message5 = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
        #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
    except:
        try:
            message5 = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        except:
            message5 = None
    
    try:
        review_info5 = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
        #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
    except:
        review_info5 = None
        
    heading5.append(header5)
    messaging5.append(message5)
    info5.append(review_info5)

Not Found
<span>Samba ADV Shoes</span>
$100
3 colors available


In [127]:
len(heading5), len(messaging5), len(info5)

(3, 3, 3)

In [128]:
heading5

['<strong class="title___1LLYl clamped___1_0yP">This shows is both fashionable and functional.</strong>',
 '<strong class="title___1LLYl clamped___1_0yP">Love this version of the samba!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP">Nice update for the OG</strong>']

In [417]:
heading5[2][46:]

'Nice update for the OG</strong>'

In [419]:
heading5_clean = []

for i,head in enumerate(heading5):
    clean_head = heading5[i][46:].replace("</strong>", "")
    heading5_clean.append(clean_head)
    
heading5_clean

['This shows is both fashionable and functional.',
 'Love this version of the samba!',
 'Nice update for the OG']

In [129]:
messaging5

['<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">It offers a lot of variety in terms of style but the width of the shoe leaves something to be desired when thinking about wearing it daily.</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">I love how this version has the suede and leather, it gives it a more textured look. I also like that the tongue isn’t as long as the traditional samba, which would irritate me. Overall really like them already!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">Bought these for my daughter as a BTS staple, and she loves them.</div><div class="read-more-btn-cont

In [422]:
messaging5_clean = []

for i,message in enumerate(messaging5):
    clean_message = message.replace('<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">', "").replace(
     '</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>',
        ""
    )
    messaging5_clean.append(clean_message)
    
len(messaging5_clean)

3

In [423]:
messaging5_clean

['It offers a lot of variety in terms of style but the width of the shoe leaves something to be desired when thinking about wearing it daily.',
 'I love how this version has the suede and leather, it gives it a more textured look. I also like that the tongue isn’t as long as the traditional samba, which would irritate me. Overall really like them already!',
 'Bought these for my daughter as a BTS staple, and she loves them.']

In [130]:
info5

[None, None, None]

In [449]:
shoe5 = pd.DataFrame(
    {"Title": [product_name5[6:].replace("</span>", "")]*len(messaging5),
     "Price":[product_price5[1:]]*len(messaging5),
     "ColoursAvailable": [product_colour5[0]]*len(messaging5_clean),
    "ReviewTopic": heading5_clean,
    "Reviews": messaging5_clean,
    "ReviewInfo": info5}
)
shoe5

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,Samba ADV Shoes,100,3,This shows is both fashionable and functional.,It offers a lot of variety in terms of style b...,None
1,Samba ADV Shoes,100,3,Love this version of the samba!,I love how this version has the suede and leat...,None
2,Samba ADV Shoes,100,3,Nice update for the OG,"Bought these for my daughter as a BTS staple, ...",None


In [141]:
#product 6
#Campus 00s Shoes

url = "https://www.adidas.com/us"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(15)

try:
    accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
    accept_button.click()
except:
    print("Accept button not found!")

search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
search.click()
search.send_keys("shoe")
search.send_keys(Keys.ENTER)

main_content = driver.find_element(By.ID, "main-content")
products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
products[23].click()

try:
    pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
    pop_up_button.click()
except NoSuchElementException:
    ("print no pop up")

try:
    driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
#<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
except:
    print("Not Found")
    
try:
    product_name6 = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_name6 = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

try:
    product_price6 = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_price6 = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item notranslate"]').get_attribute("innerHTML")
#<div class="gl-price-item notranslate">$100</div>  
try:
    product_colour6 = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
except NoSuchElementException:
    product_colour6 = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
#<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
print(product_name6)
print(product_price6)
print(product_colour6)

try:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
    review_drop_down.click()
for i in range (2):
    load_more_reviews = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="ratings-load-more"]')
    load_more_reviews.click()

    
try:
    read_mores = driver.find_elements(By.CSS_SELECTOR, '[data-auto-id="review-read-more-button"]')
    #<button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>
except:
     read_mores = driver.find_elements(By.CSS_SELECTOR, '[class="read-more___2dacS gl-link gl-label--m"]')


for reading in read_mores:
    reading.click()
    
heading6 = []
messaging6 = []
info6 = []

#actual reviews
reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
#<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
for review in reviews:
    header6 = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
    #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>
    
    try:
        message6 = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
        #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
    except:
        try:
            message6 = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        except:
            message6 = None
    
    try:
        review_info6 = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
        #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
    except:
        review_info6 = None
        
    heading6.append(header6)
    messaging6.append(message6)
    info6.append(review_info6)

Not Found
<span>Campus 00s Shoes</span>
$110
7 colors available


In [431]:
len(heading6), len(messaging6), len(info6)

(12, 12, 12)

In [143]:
heading6

['<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Runs very big</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">This shoe is completely defected and I would like</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Reminiscent of High School</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Love these shoes</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Nice</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">fits great, extremely comfortable, very happy!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">excellent shoe love the style and  color</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Love the materials</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Have 4 pairs and love them all</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">

In [433]:
heading6_clean = []

for i,head in enumerate(heading6):
    clean_head = head.replace('<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">', "").replace('</strong>',"")
    heading6_clean.append(clean_head)
    
heading6_clean

['Runs very big',
 'This shoe is completely defected and I would like',
 'Reminiscent of High School',
 'Love these shoes',
 'Nice',
 'fits great, extremely comfortable, very happy!',
 'excellent shoe love the style and  color',
 'Love the materials',
 'Have 4 pairs and love them all',
 'Super cute',
 'Love these my new favorite shoe!',
 'Totally drip shoes']

In [144]:
messaging6

['<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Runs very big. My daughter always wears a 10 1/2 and this was 2 inches too big.  Thought is was men’s size but it was not.  Still waiting for our exchange.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">This shoe is completely defected and I would like someone to call me back so I can ship the shoes back to you worst quality I’ve ever seen</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">These are super cute! They are comfy but super wide. They have the look of DCs or Etnies from the 2000s. I probably could have sized down, but after 2 kids my feet ate a half size bigger so I didn’t want to chance them being too small.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Love this shoes so cute and stylish, runs true to size.</div>',
 '<div class="clamped___1_0yP gl-vspa

In [428]:
messaging6_clean = []

for i,message in enumerate(messaging6):
    clean_message = message.replace('<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">', "").replace(
     '</div>', ""
    )
    messaging6_clean.append(clean_message)
    
len(messaging6_clean)

12

In [429]:
messaging6_clean

['Runs very big. My daughter always wears a 10 1/2 and this was 2 inches too big.  Thought is was men’s size but it was not.  Still waiting for our exchange.',
 'This shoe is completely defected and I would like someone to call me back so I can ship the shoes back to you worst quality I’ve ever seen',
 'These are super cute! They are comfy but super wide. They have the look of DCs or Etnies from the 2000s. I probably could have sized down, but after 2 kids my feet ate a half size bigger so I didn’t want to chance them being too small.',
 'Love this shoes so cute and stylish, runs true to size.',
 "It's nice shoes and I will recomend it. Comfortable very",
 'the usual expectations were met, very well fitting and very comfortable, and stylish as well. happy as always',
 'super comfortable, goes great with all styles of cloths.',
 'Great colorway. Materials are good quality with soft suede. Comfortable, as expected.',
 'I love the wide laces! Fits really well and super comfy.',
 'Very cut

In [145]:
info6

['Cindy | September 7, 2023 | Incentivized Review',
 'Vonny | September 1, 2023',
 'Anzgei904 | August 28, 2023 | Incentivized Review',
 'Robin | August 17, 2023 | Incentivized Review',
 'Grig | August 16, 2023 | Incentivized Review',
 'CBass | July 7, 2023',
 'hamac | July 6, 2023',
 'Jenn | July 5, 2023',
 'Sunfun | July 4, 2023 | Incentivized Review',
 'Dani | June 29, 2023',
 'marie1226 | June 27, 2023',
 'nsneely | June 26, 2023 | Incentivized Review']

In [450]:
shoe6 = pd.DataFrame(
    {"Title": [product_name6[6:].replace("</span>", "")]*len(messaging6),
     "Price":[product_price6[1:]]*len(messaging6),
     "ColoursAvailable": [product_colour6[0]]*len(messaging6_clean),
    "ReviewTopic": heading6_clean,
    "Reviews": messaging6_clean,
    "ReviewInfo": info6}
)
shoe6

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,Campus 00s Shoes,110,7,Runs very big,Runs very big. My daughter always wears a 10 1...,"Cindy | September 7, 2023 | Incentivized Review"
1,Campus 00s Shoes,110,7,This shoe is completely defected and I would like,This shoe is completely defected and I would l...,"Vonny | September 1, 2023"
2,Campus 00s Shoes,110,7,Reminiscent of High School,These are super cute! They are comfy but super...,"Anzgei904 | August 28, 2023 | Incentivized Review"
3,Campus 00s Shoes,110,7,Love these shoes,"Love this shoes so cute and stylish, runs true...","Robin | August 17, 2023 | Incentivized Review"
4,Campus 00s Shoes,110,7,Nice,It's nice shoes and I will recomend it. Comfor...,"Grig | August 16, 2023 | Incentivized Review"
5,Campus 00s Shoes,110,7,"fits great, extremely comfortable, very happy!","the usual expectations were met, very well fit...","CBass | July 7, 2023"
6,Campus 00s Shoes,110,7,excellent shoe love the style and color,"super comfortable, goes great with all styles ...","hamac | July 6, 2023"
7,Campus 00s Shoes,110,7,Love the materials,Great colorway. Materials are good quality wit...,"Jenn | July 5, 2023"
8,Campus 00s Shoes,110,7,Have 4 pairs and love them all,I love the wide laces! Fits really well and su...,"Sunfun | July 4, 2023 | Incentivized Review"
9,Campus 00s Shoes,110,7,Super cute,Very cute and comfy to wear just to go walking...,"Dani | June 29, 2023"


In [151]:
#product 7
#NMD_R1 Shoes

url = "https://www.adidas.com/us"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(15)

try:
    accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
    accept_button.click()
except:
    print("Accept button not found!")

search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
search.click()
search.send_keys("shoe")
search.send_keys(Keys.ENTER)

main_content = driver.find_element(By.ID, "main-content")
products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
products[24].click()

try:
    pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
    pop_up_button.click()
except NoSuchElementException:
    ("print no pop up")

try:
    driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
#<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
except:
    print("Not Found")
    
try:
    product_name7 = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_name7 = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

try:
    product_price7 = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_price7 = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item notranslate"]').get_attribute("innerHTML")
#<div class="gl-price-item notranslate">$100</div>  
try:
    product_colour7 = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
except NoSuchElementException:
    product_colour7 = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
#<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
print(product_name7)
print(product_price7)
print(product_colour7)

try:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
    review_drop_down.click()
for i in range (100):
    load_more_reviews = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="ratings-load-more"]')
    load_more_reviews.click()

    
try:
    read_mores = driver.find_elements(By.CSS_SELECTOR, '[data-auto-id="review-read-more-button"]')
    #<button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>
except:
     read_mores = driver.find_elements(By.CSS_SELECTOR, '[class="read-more___2dacS gl-link gl-label--m"]')


for reading in read_mores[:400]:
    reading.click()
    
heading7 = []
messaging7 = []
info7 = []

#actual reviews
reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
#<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
for review in reviews:
    header7 = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
    #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>
    
    try:
        message7 = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
        #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
    except:
        try:
            message7 = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        except:
            message7 = None
    
    try:
        review_info7 = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
        #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
    except:
        review_info7 = None
        
    heading7.append(header7)
    messaging7.append(message7)
    info7.append(review_info7)

Not Found
<span>NMD_R1 Shoes</span>
$150
31 colors available


In [153]:
len(heading7), len(messaging7), len(info7)

(192, 192, 192)

In [154]:
heading7

['<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Great overall buy!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Have nothing to complain about</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Greatest purchase ever</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Lightweight and comfortable running shoes.</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">As Comfy as EVER !!!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">I will never buy this again they hurt my feet!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">One of the best, fashionable, comfortable sneakers</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Very comfortable lightweight shoe!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Sweet</strong>',
 '<strong class="title___1LLYl clamped__

In [452]:
heading7_clean = []

jag1 = '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">'
jag2 = '<strong class="title___1LLYl clamped___1_0yP">'

for i,head in enumerate(heading7):
    if jag1 in head:
        clean_head = head.replace(jag1, "").replace('</strong>',"")
    else:
        clean_head = head.replace(jag2, "").replace('</strong>',"")
        
    heading7_clean.append(clean_head)
    
heading7_clean

['Great overall buy!',
 'Have nothing to complain about',
 'Greatest purchase ever',
 'Lightweight and comfortable running shoes.',
 'As Comfy as EVER !!!',
 'I will never buy this again they hurt my feet!',
 'One of the best, fashionable, comfortable sneakers',
 'Very comfortable lightweight shoe!',
 'Sweet',
 'I love it the color is perfect!',
 'As usual adidas product to do disappoint',
 'Excellent',
 'Excellent',
 'Love these!',
 'Very reliable product.',
 'Excellent feeling shoe.',
 'Not true to size',
 'Excellent',
 'Love these!',
 'Very reliable product.',
 'Excellent feeling shoe.',
 'Not true to size',
 'Excellent',
 'Love these!',
 'Very reliable product.',
 'Excellent feeling shoe.',
 'Not true to size',
 'Amazing',
 'Pretty good shoes',
 'Great product',
 'My wife loved them. Comfy and stylish',
 'Great color really nice pink',
 'Very good work shoes and great quality',
 'Love NMDs! My go to sneaker. So comfy. Love ‘em!',
 'I love this sneaker!',
 'I feel it is ok',
 'Great

In [156]:
messaging7

['<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Would purchase again  sizing was perfect, comfortable</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">The shoe is great. It feels great on my feet. The pair I had before this has lasted 6 years.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Great Price, Good Quality, Fast Shipping and Awesome deal..</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Cloud foam support is very comfortable. Will be buying in more colors</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Keep it up ADIDAS  !!!!\nA +++++×++++++++++++++++++++</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">The shoe hurts so bad they are so hard on the top of the shoe I will never buy this again</div>',
 '<div class="c

In [454]:
messaging7_clean = []

jag1 = '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">'
jag2 = '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">'
jag3 = '</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" \
class="read-more___2dacS gl-link gl-label--m">Read more</button></div>'

for i,message in enumerate(messaging7):
    if jag1 in message:
        clean_message = message.replace(jag1, "").replace('</div>', "")
        messaging7_clean.append(clean_message)
    else:
        clean_message = message.replace(jag2, "").replace(jag3, "")
        messaging7_clean.append(clean_message)
    
messaging7_clean

['Would purchase again  sizing was perfect, comfortable',
 'The shoe is great. It feels great on my feet. The pair I had before this has lasted 6 years.',
 'Great Price, Good Quality, Fast Shipping and Awesome deal..',
 'Cloud foam support is very comfortable. Will be buying in more colors',
 'Keep it up ADIDAS  !!!!\nA +++++×++++++++++++++++++++',
 'The shoe hurts so bad they are so hard on the top of the shoe I will never buy this again',
 'A bit pricey, but my number one go to for fashion and comfort.',
 'Very comfortable and lightweight shoe. Feels great, very sharp appearance,',
 'All black shoes, really comfortable, fit very nicely',
 'The color is a must! I love it! If you like use colors this is the shoe and its soo comfy!',
 "Adidas for life, definitely my favorite brand that I'm always excited about getting new gear from",
 'Very comfortable. I have them in several pairs in various colors',
 'Very comfortable. I have them in several pairs in various colors',
 'Great standard 

In [157]:
info7

['Slab88 | September 13, 2023',
 'Matt | September 13, 2023',
 'NMDs | September 12, 2023',
 'Tj22 | September 12, 2023',
 'Gestos34 | September 12, 2023',
 'Betty83 | September 11, 2023',
 'Haley | September 10, 2023',
 'Breed48 | September 9, 2023',
 '6jgff | September 6, 2023',
 'Evea | September 6, 2023',
 'CharlieC003 | September 5, 2023',
 'Nabisco721 | September 5, 2023 | Incentivized Review',
 'Nabisco721 | September 5, 2023 | Incentivized Review',
 'Shanalama | September 5, 2023',
 'TedBaq888 | September 5, 2023 | Incentivized Review',
 'Aristy26 | September 5, 2023 | Incentivized Review',
 'Britt87 | September 4, 2023',
 'Nabisco721 | September 5, 2023 | Incentivized Review',
 'Shanalama | September 5, 2023',
 'TedBaq888 | September 5, 2023 | Incentivized Review',
 'Aristy26 | September 5, 2023 | Incentivized Review',
 'Britt87 | September 4, 2023',
 'Nabisco721 | September 5, 2023 | Incentivized Review',
 'Shanalama | September 5, 2023',
 'TedBaq888 | September 5, 2023 | Inc

In [455]:
shoe7 = pd.DataFrame(
    {"Title": [product_name7[6:].replace("</span>", "")]*len(messaging7),
     "Price":[product_price6[1:]]*len(messaging7),
     "ColoursAvailable": [product_colour7[0]]*len(messaging7_clean),
    "ReviewTopic": heading7_clean,
    "Reviews": messaging7_clean,
    "ReviewInfo": info7}
)
shoe7

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,NMD_R1 Shoes,110,3,Great overall buy!,"Would purchase again sizing was perfect, comf...","Slab88 | September 13, 2023"
1,NMD_R1 Shoes,110,3,Have nothing to complain about,The shoe is great. It feels great on my feet. ...,"Matt | September 13, 2023"
2,NMD_R1 Shoes,110,3,Greatest purchase ever,"Great Price, Good Quality, Fast Shipping and A...","NMDs | September 12, 2023"
3,NMD_R1 Shoes,110,3,Lightweight and comfortable running shoes.,Cloud foam support is very comfortable. Will b...,"Tj22 | September 12, 2023"
4,NMD_R1 Shoes,110,3,As Comfy as EVER !!!,Keep it up ADIDAS !!!!\nA +++++×+++++++++++++...,"Gestos34 | September 12, 2023"
...,...,...,...,...,...,...
187,NMD_R1 Shoes,110,3,Best purchase,"In love with the color of this shoes, the fill...",None
188,NMD_R1 Shoes,110,3,The best,Great Colab shoes unique designed. The box pre...,None
189,NMD_R1 Shoes,110,3,The perfect black sneaker!,"The perfect black sneaker, which made me a new...",None
190,NMD_R1 Shoes,110,3,Very cool but size down 0.5,Super cool looking. Massive amounts of boost -...,None


In [242]:
#product 8
#NMD_R1 Shoe
url = "https://www.adidas.com/us"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(15)

try:
    accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
    accept_button.click()
except:
    print("Accept button not found!")

search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
search.click()
search.send_keys("shoe")
search.send_keys(Keys.ENTER)

main_content = driver.find_element(By.ID, "main-content")
products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
products[25].click()

try:
    pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
    pop_up_button.click()
except NoSuchElementException:
    ("print no pop up")

try:
    driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
#<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
except:
    print("Not Found")
    
try:
    product_name8 = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_name8 = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

try:
    product_price8 = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_price8 = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item notranslate"]').get_attribute("innerHTML")
#<div class="gl-price-item notranslate">$100</div>  
try:
    product_colour8 = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
except NoSuchElementException:
    product_colour8 = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
#<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
print(product_name8)
print(product_price8)
print(product_colour8)

try:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
    review_drop_down.click()
for i in range (100):
    load_more_reviews = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="ratings-load-more"]')
    load_more_reviews.click()

    
try:
    read_mores = driver.find_elements(By.CSS_SELECTOR, '[data-auto-id="review-read-more-button"]')
    #<button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>
except:
     read_mores = driver.find_elements(By.CSS_SELECTOR, '[class="read-more___2dacS gl-link gl-label--m"]')


for reading in read_mores[:400]:
    reading.click()
    
heading8 = []
messaging8 = []
info8 = []

#actual reviews
reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
#<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
for review in reviews:
    header8 = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
    #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>
    
    try:
        message8 = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
        #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
    except:
        try:
            message8 = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        except:
            message8 = None
    
    try:
        review_info8 = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
        #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
    except:
        review_info8 = None
        
    heading8.append(header8)
    messaging8.append(message8)
    info8.append(review_info8)

Not Found
<span>NMD_R1 Shoes</span>
$150
33 colors available


In [243]:
len(heading8), len(messaging8), len(info8)

(282, 282, 282)

In [178]:
heading8

['<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Great overall buy!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Have nothing to complain about</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Greatest purchase ever</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Lightweight and comfortable running shoes.</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">As Comfy as EVER !!!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">I will never buy this again they hurt my feet!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">One of the best, fashionable, comfortable sneakers</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">One of the best, fashionable, comfortable sneakers</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Very comfortable lightweight shoe!</stron

In [459]:
heading8_clean = []

for i,head in enumerate(heading8):
    if '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">' in head:
        clean_head = head.replace('<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">', "").replace('</strong>',"")
    else:
        clean_head = head.replace('<strong class="title___1LLYl clamped___1_0yP">', "").replace('</strong>',"")
    heading8_clean.append(clean_head)
    
len(heading8_clean)

282

In [462]:
messaging8

['<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Would purchase again  sizing was perfect, comfortable</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">The shoe is great. It feels great on my feet. The pair I had before this has lasted 6 years.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Great Price, Good Quality, Fast Shipping and Awesome deal..</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Left really bad blisters on the back of my heels they take awhile to break in. Have to wear higher socks until you get used to them other than that I have always loved this style shoe.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Cloud foam support is very comfortable. Will be buying in more colors</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom 

In [465]:
messaging8_clean = []

for i,message in enumerate(messaging8):
    if  '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">' in message:
        clean_message = message.replace('<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">', "").replace(
     '</div>', ""
    )
    else:
         clean_message = message.replace('<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">', "").replace(
     '</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>', ""
    )
    messaging8_clean.append(clean_message)
    
len(messaging8_clean)

282

In [466]:
messaging8_clean

['Would purchase again  sizing was perfect, comfortable',
 'The shoe is great. It feels great on my feet. The pair I had before this has lasted 6 years.',
 'Great Price, Good Quality, Fast Shipping and Awesome deal..',
 'Left really bad blisters on the back of my heels they take awhile to break in. Have to wear higher socks until you get used to them other than that I have always loved this style shoe.',
 'Cloud foam support is very comfortable. Will be buying in more colors',
 'Keep it up ADIDAS  !!!!\nA +++++×++++++++++++++++++++',
 'The shoe hurts so bad they are so hard on the top of the shoe I will never buy this again',
 'Very comfortable and lightweight shoe. Feels great, very sharp appearance,',
 'All black shoes, really comfortable, fit very nicely',
 'The color is a must! I love it! If you like use colors this is the shoe and its soo comfy!',
 "Adidas for life, definitely my favorite brand that I'm always excited about getting new gear from",
 'Very comfortable. I have them i

In [180]:
info8

['Slab88 | September 13, 2023',
 'Matt | September 13, 2023',
 'NMDs | September 12, 2023',
 'Tj22 | September 12, 2023',
 'Gestos34 | September 12, 2023',
 'Betty83 | September 11, 2023',
 'Haley | September 10, 2023',
 'Haley | September 10, 2023',
 'Breed48 | September 9, 2023',
 '6jgff | September 6, 2023',
 'Evea | September 6, 2023',
 'CharlieC003 | September 5, 2023',
 'Haley | September 10, 2023',
 'Breed48 | September 9, 2023',
 '6jgff | September 6, 2023',
 'Evea | September 6, 2023',
 'CharlieC003 | September 5, 2023',
 'Nabisco721 | September 5, 2023 | Incentivized Review',
 'Shanalama | September 5, 2023',
 'TedBaq888 | September 5, 2023 | Incentivized Review',
 'Aristy26 | September 5, 2023 | Incentivized Review',
 'Britt87 | September 4, 2023',
 'Britt87 | September 4, 2023',
 'Dp51 | September 4, 2023',
 'john | September 4, 2023',
 'Omar | September 3, 2023',
 'Adidasfreak | September 3, 2023',
 'Britt87 | September 4, 2023',
 'Dp51 | September 4, 2023',
 'john | Septe

In [467]:
shoe8 = pd.DataFrame(
    {"Title": [product_name8[6:].replace("</span>", "")]*len(messaging8),
     "Price":[product_price8[1:]]*len(messaging8),
     "ColoursAvailable": [product_colour8[0]]*len(messaging8_clean),
    "ReviewTopic": heading8_clean,
    "Reviews": messaging8_clean,
    "ReviewInfo": info8}
)
shoe8

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,NMD_R1 Shoes,150,3,Great overall buy!,"Would purchase again sizing was perfect, comf...","Slab88 | September 13, 2023"
1,NMD_R1 Shoes,150,3,Have nothing to complain about,The shoe is great. It feels great on my feet. ...,"Matt | September 13, 2023"
2,NMD_R1 Shoes,150,3,Greatest purchase ever,"Great Price, Good Quality, Fast Shipping and A...","NMDs | September 12, 2023"
3,NMD_R1 Shoes,150,3,Beware will leave blisters on the back of heels.,Left really bad blisters on the back of my hee...,"JayLynn | September 12, 2023"
4,NMD_R1 Shoes,150,3,Lightweight and comfortable running shoes.,Cloud foam support is very comfortable. Will b...,"Tj22 | September 12, 2023"
...,...,...,...,...,...,...
277,NMD_R1 Shoes,150,3,It’s really nice and comfortable,The shipping was so fast and the quality good ...,None
278,NMD_R1 Shoes,150,3,Best purchase ever,Love NMD'S. My absolute favorite pair of shoes...,None
279,NMD_R1 Shoes,150,3,Eye catching and running a half size big.,They run big. I bought my regular size and had...,None
280,NMD_R1 Shoes,150,3,Love NMDS they’re soo comfortable!,I wear my NMDS for work and I’m on my feet for...,None


In [187]:
#product 9
#Samba Classic
url = "https://www.adidas.com/us"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(15)

try:
    accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
    accept_button.click()
except:
    print("Accept button not found!")

search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
search.click()
search.send_keys("shoe")
search.send_keys(Keys.ENTER)

main_content = driver.find_element(By.ID, "main-content")
products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
products[26].click()

try:
    pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
    pop_up_button.click()
except NoSuchElementException:
    ("print no pop up")

try:
    driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
#<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
except:
    print("Not Found")
    
try:
    product_name9 = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_name9 = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

try:
    product_price9 = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_price9 = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item notranslate"]').get_attribute("innerHTML")
#<div class="gl-price-item notranslate">$100</div>  
try:
    product_colour9 = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
except NoSuchElementException:
    product_colour9 = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
#<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
print(product_name9)
print(product_price9)
print(product_colour9)

try:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
    review_drop_down.click()
for i in range (100):
    load_more_reviews = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="ratings-load-more"]')
    load_more_reviews.click()

    
try:
    read_mores = driver.find_elements(By.CSS_SELECTOR, '[data-auto-id="review-read-more-button"]')
    #<button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>
except:
     read_mores = driver.find_elements(By.CSS_SELECTOR, '[class="read-more___2dacS gl-link gl-label--m"]')


for reading in read_mores[:400]:
    reading.click()
    
heading9 = []
messaging9 = []
info9 = []

#actual reviews
reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
#<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
for review in reviews:
    header9 = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
    #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>
    
    try:
        message9 = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
        #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
    except:
        try:
            message9 = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        except:
            message9 = None
    
    try:
        review_info9 = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
        #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
    except:
        review_info9 = None
        
    heading9.append(header9)
    messaging9.append(message9)
    info9.append(review_info9)

Not Found
<span>Samba Classic</span>
$90
2 colors available


In [188]:
len(heading9), len(messaging9), len(info9)

(217, 217, 217)

In [189]:
heading9

['<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">nice shoes, classic and trendy</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Solid fit, rethink twice about going 1/2 a size up</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">love these</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Vey fashionable shoes</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Timeless design</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Great product</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Great price for these sneakers!!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Vey fashionable shoes</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Timeless design</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Great product</strong>',
 '<strong 

In [468]:
heading9_clean = []

for i,head in enumerate(heading9):
    if '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">' in head:
        clean_head = head.replace('<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">', "").replace('</strong>',"")
        heading9_clean.append(clean_head)
    else:
        clean_head = head.replace('<strong class="title___1LLYl clamped___1_0yP">', "").replace('</strong>',"")
        heading9_clean.append(clean_head)
heading9_clean

['nice shoes, classic and trendy',
 'Solid fit, rethink twice about going 1/2 a size up',
 'love these',
 'Vey fashionable shoes',
 'Timeless design',
 'Great product',
 'Great price for these sneakers!!',
 'Vey fashionable shoes',
 'Timeless design',
 'Great product',
 'Great price for these sneakers!!',
 'A “must” buy',
 'Vey fashionable shoes',
 'Timeless design',
 'Great product',
 'Great price for these sneakers!!',
 'A “must” buy',
 'Fits true to size very sleek and stylish',
 'Chic &amp; comfortable',
 'Reliable and a favorite shoe since I was a kid!',
 'Great for daily use!',
 'Stylish &amp; comfortable',
 'Honestly one of the best product',
 'Amazing purchase super easy to clean and durable',
 'It’s a good everyday shoe to wear',
 'Best purchase ever.',
 'Best purchase ever.',
 'Best purchase ever.',
 'Best purchase ever.',
 'Good product to buy.',
 'Good product to buy.',
 'Love them',
 'Love the product!',
 'Timeless classic!',
 'Decent for indoor soccer',
 'Great Classic Ol

In [190]:
messaging9

['<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">bought these for my brother and he likes them a lot. nice outfits and i get sister of the year!</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Pretty good shoe, really big fan of the samba brand.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">love these they are so comfy and cute, i love wearing them, they are my everyday shoe i literally cannot take them off</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">I like these shoes a lot my only complaint is that they’re not super comfortable cause they were a bit narrow for my feet. And that they only have samba og for men and children but not women. Just wish they were a bit more inclusive.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Can never go wrong with this shoe wi

In [470]:
messaging9_clean = []

for i,message in enumerate(messaging9):
    if '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">' in message:
        clean_message = message.replace('<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">', "").replace(
     '</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>', ""
    )
        messaging9_clean.append(clean_message)
    else:
        clean_message = message.replace('<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">', "").replace(
     '</div>', ""
    )
        messaging9_clean.append(clean_message)
    
len(messaging9_clean)

217

In [471]:
messaging9_clean

['bought these for my brother and he likes them a lot. nice outfits and i get sister of the year!',
 'Pretty good shoe, really big fan of the samba brand.',
 'love these they are so comfy and cute, i love wearing them, they are my everyday shoe i literally cannot take them off',
 'I like these shoes a lot my only complaint is that they’re not super comfortable cause they were a bit narrow for my feet. And that they only have samba og for men and children but not women. Just wish they were a bit more inclusive.',
 'Can never go wrong with this shoe with any outfit, this design has stood the test of time and generations',
 'Positive Experience and great style. Very comfortable',
 'Comfortable and I can wear them with anything. The colors go with any color.',
 'I like these shoes a lot my only complaint is that they’re not super comfortable cause they were a bit narrow for my feet. And that they only have samba og for men and children but not women. Just wish they were a bit more inclusiv

In [191]:
info9

['lucygoosey | September 13, 2023',
 'Tuttywutty | September 12, 2023',
 'veve | September 12, 2023',
 'Jamie | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'LuvthesepantZ | September 11, 2023',
 'Maverick | September 11, 2023 | Incentivized Review',
 'Cocoa | September 10, 2023 | Verified Purchaser | Incentivized Review',
 'Jamie | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'LuvthesepantZ | September 11, 2023',
 'Maverick | September 11, 2023 | Incentivized Review',
 'Cocoa | September 10, 2023 | Verified Purchaser | Incentivized Review',
 'Wiseguy83 | September 10, 2023',
 'Jamie | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'LuvthesepantZ | September 11, 2023',
 'Maverick | September 11, 2023 | Incentivized Review',
 'Cocoa | September 10, 2023 | Verified Purchaser | Incentivized Review',
 'Wiseguy83 | September 10, 2023',
 'Kash | September 9, 2023',
 'DanS | September 9, 2023 | Incentivized Review',
 'Jj12 | September 

In [472]:
shoe9 = pd.DataFrame(
    {"Title": [product_name9[6:].replace("</span>", "")]*len(messaging9),
     "Price":[product_price9[1:]]*len(messaging9),
     "ColoursAvailable": [product_colour9[0]]*len(messaging9_clean),
    "ReviewTopic": heading9_clean,
    "Reviews": messaging9_clean,
    "ReviewInfo": info9}
)
shoe9

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,Samba Classic,90,2,"nice shoes, classic and trendy",bought these for my brother and he likes them ...,"lucygoosey | September 13, 2023"
1,Samba Classic,90,2,"Solid fit, rethink twice about going 1/2 a siz...","Pretty good shoe, really big fan of the samba ...","Tuttywutty | September 12, 2023"
2,Samba Classic,90,2,love these,"love these they are so comfy and cute, i love ...","veve | September 12, 2023"
3,Samba Classic,90,2,Vey fashionable shoes,I like these shoes a lot my only complaint is ...,"Jamie | September 12, 2023 | Verified Purchase..."
4,Samba Classic,90,2,Timeless design,Can never go wrong with this shoe with any out...,"LuvthesepantZ | September 11, 2023"
...,...,...,...,...,...,...
212,Samba Classic,90,2,Cool model and very good quality,"I like how the tongue is, and the employees we...",None
213,Samba Classic,90,2,"Slick shoes, really enjoy them",I’m happy a got ‘em! Chill to dance in too!! G...,None
214,Samba Classic,90,2,Good everyday shoe,Good everyday shoe pt.2 \nGetting a pair is wo...,None
215,Samba Classic,90,2,Great shoe.,"Good quality, and amazing retro style, perfect...",None


In [255]:
#product 10
#NMD_S1 Shoes
url = "https://www.adidas.com/us"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(15)

try:
    accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
    accept_button.click()
except:
    print("Accept button not found!")

search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
search.click()
search.send_keys("shoe")
search.send_keys(Keys.ENTER)

main_content = driver.find_element(By.ID, "main-content")
products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
products[28].click()

try:
    pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
    pop_up_button.click()
except NoSuchElementException:
    ("print no pop up")

try:
    driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
#<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
except:
    print("Not Found")
    
try:
    product_name10 = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_name10 = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

try:
    product_price10 = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_price10 = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item gl-price-item--sale notranslate"]').get_attribute("innerHTML")
#<div class="gl-price-item notranslate">$100</div>  
#<div class="gl-price-item gl-price-item--sale notranslate">$80</div>
try:
    product_colour10 = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
except NoSuchElementException:
    product_colour10 = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
#<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
print(product_name10)
print(product_price10)
print(product_colour10)

try:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
    review_drop_down.click()
for i in range (10):
    load_more_reviews = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="ratings-load-more"]')
    load_more_reviews.click()

    
try:
    read_mores = driver.find_elements(By.CSS_SELECTOR, '[data-auto-id="ratings-load-more"]')
    #<button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>
    #<button class="gl-link gl-body--s gl-no-margin-bottom" data-auto-id="ratings-load-more">Read more reviews</button>
except:
     read_mores = driver.find_elements(By.CSS_SELECTOR, '[class="read-more___2dacS gl-link gl-label--m"]')


for reading in read_mores[:30]:
    reading.click()
    
heading10 = []
messaging10 = []
info10 = []

#actual reviews
reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
#<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
for review in reviews:
    header10 = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
    #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>
    
    try:
        message10 = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
        #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
    except:
        try:
            message10 = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        except:
            message10 = None
    
    try:
        review_info10 = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
        #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
    except:
        review_info10 = None
        
    heading10.append(header10)
    messaging10.append(message10)
    info10.append(review_info10)

Not Found
<span>NMD_S1 Shoes</span>
$80
9 colors available


In [256]:
len(heading10), len(messaging10), len(info10)

(32, 32, 32)

In [257]:
heading10

['<strong class="title___1LLYl clamped___1_0yP">Great looking shoe and super comfortable</strong>',
 '<strong class="title___1LLYl clamped___1_0yP">My favorite shoes that l bought from Adidas</strong>',
 '<strong class="title___1LLYl clamped___1_0yP">Would but again</strong>',
 '<strong class="title___1LLYl clamped___1_0yP">Like walking on clouds, but it\'s a bit heavy</strong>',
 '<strong class="title___1LLYl clamped___1_0yP">Beauty is pain.</strong>',
 '<strong class="title___1LLYl clamped___1_0yP">One of my best purchases!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP">the NMD S1 is probably the best NMD version ever</strong>',
 '<strong class="title___1LLYl clamped___1_0yP">A great sneaker. Very comfortable and stylish</strong>',
 '<strong class="title___1LLYl clamped___1_0yP">Quality Matters</strong>',
 '<strong class="title___1LLYl clamped___1_0yP">2nd pair of nmd s1</strong>',
 '<strong class="title___1LLYl clamped___1_0yP">NMD1 all black will be most likely my new f

In [473]:
heading10_clean = []

for i,head in enumerate(heading10):
    clean_head = head.replace('<strong class="title___1LLYl clamped___1_0yP">', "").replace('</strong>',"")
    heading10_clean.append(clean_head)
heading10_clean

['Great looking shoe and super comfortable',
 'My favorite shoes that l bought from Adidas',
 'Would but again',
 "Like walking on clouds, but it's a bit heavy",
 'Beauty is pain.',
 'One of my best purchases!',
 'the NMD S1 is probably the best NMD version ever',
 'A great sneaker. Very comfortable and stylish',
 'Quality Matters',
 '2nd pair of nmd s1',
 'NMD1 all black will be most likely my new favorite',
 'Very happy with quality and pricing.',
 'A great sneaker. Very comfortable and stylish',
 'Quality Matters',
 '2nd pair of nmd s1',
 'NMD1 all black will be most likely my new favorite',
 'Very happy with quality and pricing.',
 'A great sneaker. Very comfortable and stylish',
 'Quality Matters',
 '2nd pair of nmd s1',
 'NMD1 all black will be most likely my new favorite',
 'Very happy with quality and pricing.',
 '2nd pair of nmd s1',
 'NMD1 all black will be most likely my new favorite',
 'Very happy with quality and pricing.',
 'Perfect',
 'Very stylish and comfortable.',
 'V

In [258]:
messaging10

['<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">Best looking NMD in my opinion, great quality and a fabulous design</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">Great product, real size ,perfect width l would definitely recommend it</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">I likey … always wanted a pair and they dropped so low I had to</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">I love the

In [475]:
messaging10_clean = []

for i,message in enumerate(messaging10):
    clean_message = message.replace('<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">', "").replace(
     '</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>', ""
    )
    messaging10_clean.append(clean_message)
    
messaging10_clean

['Best looking NMD in my opinion, great quality and a fabulous design',
 'Great product, real size ,perfect width l would definitely recommend it',
 'I likey … always wanted a pair and they dropped so low I had to',
 "I love these shoes! I bought a half size down (US size), but they fit well. The NMS S1s feel like walking on clouds, but the you can feel the weight of the shoe. But I've worn them for hrs and dint feel much fatigue.",
 'Great shoes for fashion. Unfortunately, this brings "beauty is pain" back in fashion. Don\'t get me wrong. These are great looking sneakers. It\'s just unfortunate that I would experience discomfort in the first 2 hours of wearing them, and outright pain when I wore them for a full work day the one time I did.',
 'Love the style on these kicks!! I work 10 hours on feet and would recommend. I have them in black and grey. Will purchase more colors.',
 'Order an 1/2 down for the perfect fit even for the wide footer',
 'Very stylish, comfortable and well made

In [259]:
info10

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [476]:
shoe10 = pd.DataFrame(
    {"Title": [product_name10[6:].replace("</span>", "")]*len(messaging10),
     "Price":[product_price9[1:]]*len(messaging10),
     "ColoursAvailable": [product_colour10[0]]*len(messaging10_clean),
    "ReviewTopic": heading10_clean,
    "Reviews": messaging10_clean,
    "ReviewInfo": info10}
)
shoe10

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,NMD_S1 Shoes,90,9,Great looking shoe and super comfortable,"Best looking NMD in my opinion, great quality ...",None
1,NMD_S1 Shoes,90,9,My favorite shoes that l bought from Adidas,"Great product, real size ,perfect width l woul...",None
2,NMD_S1 Shoes,90,9,Would but again,I likey … always wanted a pair and they droppe...,None
3,NMD_S1 Shoes,90,9,"Like walking on clouds, but it's a bit heavy",I love these shoes! I bought a half size down ...,None
4,NMD_S1 Shoes,90,9,Beauty is pain.,"Great shoes for fashion. Unfortunately, this b...",None
5,NMD_S1 Shoes,90,9,One of my best purchases!,Love the style on these kicks!! I work 10 hour...,None
6,NMD_S1 Shoes,90,9,the NMD S1 is probably the best NMD version ever,Order an 1/2 down for the perfect fit even for...,None
7,NMD_S1 Shoes,90,9,A great sneaker. Very comfortable and stylish,"Very stylish, comfortable and well made. It’s ...",None
8,NMD_S1 Shoes,90,9,Quality Matters,"Great Shoes, very comfortable and cushy. I wo...",None
9,NMD_S1 Shoes,90,9,2nd pair of nmd s1,I love the style. I'm always on the lookout fo...,None


In [250]:
#product 11
#Nizza Platform Shoes
url = "https://www.adidas.com/us"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(45)

try:
    accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
    accept_button.click()
except:
    print("Accept button not found!")

search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
search.click()
search.send_keys("shoe")
search.send_keys(Keys.ENTER)

main_content = driver.find_element(By.ID, "main-content")
products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
products[18].click()

try:
    pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
    pop_up_button.click()
except NoSuchElementException:
    ("print no pop up")

try:
    driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
#<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
except:
    print("Not Found")
    
try:
    product_name11 = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_name11 = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

try:
    product_price11 = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_price11 = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item notranslate"]').get_attribute("innerHTML")
#<div class="gl-price-item notranslate">$100</div>  
try:
    product_colour11 = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
except NoSuchElementException:
    product_colour11 = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
#<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
print(product_name11)
print(product_price11)
print(product_colour11)

try:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
    review_drop_down.click()
for i in range (100):
    load_more_reviews = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="ratings-load-more"]')
    load_more_reviews.click()

    
try:
    read_mores = driver.find_elements(By.CSS_SELECTOR, '[data-auto-id="review-read-more-button"]')
    #<button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>
except:
     read_mores = driver.find_elements(By.CSS_SELECTOR, '[class="read-more___2dacS gl-link gl-label--m"]')


for reading in read_mores[:400]:
    reading.click()
    
heading11 = []
messaging11 = []
info11 = []

#actual reviews
reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
#<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
for review in reviews:
    header11 = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
    #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>
    
    try:
        message11 = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
        #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
    except:
        try:
            message11 = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        except:
            message11 = None
    
    try:
        review_info11 = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
        #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
    except:
        review_info11 = None
        
    heading11.append(header11)
    messaging11.append(message11)
    info11.append(review_info11)

Not Found
<span>Nizza Platform Shoes</span>
$75
6 colors available


In [251]:
len(heading11), len(messaging11), len(info11)

(212, 212, 212)

In [252]:
heading11

['<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Best pair of sneakers EVER!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">The size is too big for actual one</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Cute and versatile shoe</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Good buy!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Perfect fit &amp; oh so cute!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Third pair!!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Love them so much.</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">I absolutely love them</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Runs small. Size up.</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Great purchase</strong>',
 '<strong class="title

In [478]:
heading11_clean = []

for i,head in enumerate(heading11):
    if '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">' in head:
        clean_head = head.replace('<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">', "").replace('</strong>',"")
        heading11_clean.append(clean_head)
    else:
        clean_head = head.replace('<strong class="title___1LLYl clamped___1_0yP">', "").replace('</strong>',"")
        heading11_clean.append(clean_head)
heading11_clean

['Best pair of sneakers EVER!',
 'The size is too big for actual one',
 'Cute and versatile shoe',
 'Good buy!',
 'Perfect fit &amp; oh so cute!',
 'Third pair!!',
 'Love them so much.',
 'I absolutely love them',
 'Runs small. Size up.',
 'Great purchase',
 'good platforms',
 'I love them!',
 'I absolutely love them',
 'Runs small. Size up.',
 'Great purchase',
 'good platforms',
 'I love them!',
 'Ran extremely too big!',
 'This is the most comfortable casual sneaker I have',
 'When people comment on my shoes I get happy',
 'Awesome shoes!! Platform is not too high for wear',
 'One of the best purchases',
 'Great buy!',
 'Glad I finally got them!',
 'I love these shoes. They just elevate your look.',
 'Gorgeous kicks at a very reasonable price!',
 'Stylish and cute ☺️',
 'Great buy!',
 'Glad I finally got them!',
 'I love these shoes. They just elevate your look.',
 'Gorgeous kicks at a very reasonable price!',
 'Stylish and cute ☺️',
 'Great buy!',
 'Glad I finally got them!',
 'I l

In [253]:
messaging11

['<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">My favorite of all time! So comfortable and look amazing!</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Love the way of this but too wide too loose . Looks like one size larger.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Love the look of this shoe, super cute and goes with everything. The only reason I gave one star less is because the shoe slides off a bit at the heel (I don\'t need a smaller size, it would be too small). You just have to make sure to tie your laces tight and it should be good. I also had to size down my normal US size.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Not terribly comfortable out of the box. Need some breaking in. But otherwise exactly what she was looking for</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-marg

In [479]:
messaging11_clean = []

for i,message in enumerate(messaging11):
    if '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">' in message:
        clean_message = message.replace('<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">', "").replace(
     '</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>', ""
    )
        messaging11_clean.append(clean_message)
    else:
        clean_message = message.replace('<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">', "").replace(
     '</div>', ""
    )
        messaging11_clean.append(clean_message)
    
len(messaging11_clean)

212

In [481]:
messaging11_clean

['My favorite of all time! So comfortable and look amazing!',
 'Love the way of this but too wide too loose . Looks like one size larger.',
 "Love the look of this shoe, super cute and goes with everything. The only reason I gave one star less is because the shoe slides off a bit at the heel (I don't need a smaller size, it would be too small). You just have to make sure to tie your laces tight and it should be good. I also had to size down my normal US size.",
 'Not terribly comfortable out of the box. Need some breaking in. But otherwise exactly what she was looking for',
 'These are my 2nd pair of Nizza platforms and I LOVE them! Comfortable, and stylish! The colors are so feminine and  soft. Many compliments!',
 'Very comfortable and stylish. This is a great staple to have and versatile with pants or a sundress!! Love it!!',
 'I love the platform on these. They are a stylish show.',
 '10000000/10 would recommend. Some of the best that I have received',
 'I sized up still too small.

In [254]:
info11

['Penelope | September 14, 2023 | Verified Purchaser | Incentivized Review',
 'Lily | September 14, 2023',
 'Eed3 | September 13, 2023',
 'ADro | September 13, 2023 | Verified Purchaser | Incentivized Review',
 'QueenB | September 13, 2023 | Incentivized Review',
 'NEGG13 | September 12, 2023',
 'Shesn2tony | September 12, 2023 | Verified Purchaser | Incentivized Review',
 'DariusShakur | September 11, 2023',
 'Queen | September 11, 2023',
 'BB97 | September 10, 2023 | Verified Purchaser | Incentivized Review',
 'meg99 | September 10, 2023 | Verified Purchaser | Incentivized Review',
 'Naly | September 10, 2023',
 'DariusShakur | September 11, 2023',
 'Queen | September 11, 2023',
 'BB97 | September 10, 2023 | Verified Purchaser | Incentivized Review',
 'meg99 | September 10, 2023 | Verified Purchaser | Incentivized Review',
 'Naly | September 10, 2023',
 'Ronnie | September 10, 2023',
 'PavlovsDog | September 10, 2023',
 'ENYG | September 10, 2023 | Verified Purchaser | Incentivized R

In [482]:
shoe11 = pd.DataFrame(
    {"Title": [product_name11[6:].replace("</span>", "")]*len(messaging11),
     "Price":[product_price11[1:]]*len(messaging11),
     "ColoursAvailable": [product_colour11[0]]*len(messaging11_clean),
    "ReviewTopic": heading11_clean,
    "Reviews": messaging11_clean,
    "ReviewInfo": info11}
)
shoe11

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,Nizza Platform Shoes,75,6,Best pair of sneakers EVER!,My favorite of all time! So comfortable and lo...,"Penelope | September 14, 2023 | Verified Purch..."
1,Nizza Platform Shoes,75,6,The size is too big for actual one,Love the way of this but too wide too loose . ...,"Lily | September 14, 2023"
2,Nizza Platform Shoes,75,6,Cute and versatile shoe,"Love the look of this shoe, super cute and goe...","Eed3 | September 13, 2023"
3,Nizza Platform Shoes,75,6,Good buy!,Not terribly comfortable out of the box. Need ...,"ADro | September 13, 2023 | Verified Purchaser..."
4,Nizza Platform Shoes,75,6,Perfect fit &amp; oh so cute!,These are my 2nd pair of Nizza platforms and I...,"QueenB | September 13, 2023 | Incentivized Review"
...,...,...,...,...,...,...
207,Nizza Platform Shoes,75,6,Love platforms,It goes with everything! I bought it for skirt...,None
208,Nizza Platform Shoes,75,6,looks good and comfortable,trainers are comfy and stylish but unfortunate...,None
209,Nizza Platform Shoes,75,6,cute,cute trainers but a little on the narrow side ...,None
210,Nizza Platform Shoes,75,6,Satisfactory,It looks good on my wife especially on a short...,None


In [228]:
#product 12
#Adifom Stan Smith Mule Shoes
url = "https://www.adidas.com/us"
driver = webdriver.Chrome()
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(15)

try:
    accept_button = driver.find_element(By.ID, "glass-gdpr-default-consent-accept-button")
    accept_button.click()
except:
    print("Accept button not found!")

search = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="searchinput-desktop"]')
search.click()
search.send_keys("shoe")
search.send_keys(Keys.ENTER)

main_content = driver.find_element(By.ID, "main-content")
products = main_content.find_elements(By.CSS_SELECTOR, '[class="grid-item"]')
products[7].click()

try:
    pop_up_button = driver.find_element(By.CSS_SELECTOR, 'button[name="account-portal-close"]')
    pop_up_button.click()
except NoSuchElementException:
    ("print no pop up")

try:
    driver.find_element(By.CSS_SELECTOR, '[style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"]')
#<div style="position: absolute; top: 0px; left: 0px; width: 30px; height: 30px; overflow: hidden; display: block;"><img src="https://siteintercept.qualtrics.com/static/q-siteintercept/~/img/svg-close-btn-black-1.svg"></div>
except:
    print("Not Found")
    
try:
    product_name12 = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="product-title"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_name12 = driver.find_element(By.CSS_SELECTOR, '[data-testid="product-title"]').get_attribute("innerHTML")

try:
    product_price12 = driver.find_element(By.CSS_SELECTOR, '[id="gl-price-item"]').get_attribute("innerHTML")
except NoSuchElementException:
    product_price12 = driver.find_element(By.CSS_SELECTOR, '[class="gl-price-item notranslate"]').get_attribute("innerHTML")
#<div class="gl-price-item notranslate">$100</div>  
try:
    product_colour12 = driver.find_element(By.CSS_SELECTOR, "[id=available-colors-label]").get_attribute("innerHTML")
except NoSuchElementException:
    product_colour12 = driver.find_element(By.CSS_SELECTOR, '[class="skip-link___3E77n"]').get_attribute("innerHTML")
#<a class="skip-link___3E77n" href="#">Skip to add to bag</a>   
print(product_name12)
print(product_price12)
print(product_colour12)

try:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[data-testid="dropdown"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.CSS_SELECTOR, '[role="img"]')
    review_drop_down.click()
except NoSuchElementException:
    review_drop_down = driver.find_element(By.ID, 'navigation-target-reviews')
    review_drop_down.click()
for i in range (50):
    load_more_reviews = driver.find_element(By.CSS_SELECTOR, '[data-auto-id="ratings-load-more"]')
    load_more_reviews.click()

    
try:
    read_mores = driver.find_elements(By.CSS_SELECTOR, '[data-auto-id="review-read-more-button"]')
    #<button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button>
except:
     read_mores = driver.find_elements(By.CSS_SELECTOR, '[class="read-more___2dacS gl-link gl-label--m"]')


for reading in read_mores[:200]:
    reading.click()
    
heading12 = []
messaging12 = []
info12 = []

#actual reviews
reviews = driver.find_elements(By.CSS_SELECTOR, '[class="review___2sQiC gl-vspace-bpall-medium"]')
#<article class="review___2sQiC gl-vspace-bpall-medium"><div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Nice quality shoes</strong></div><div class="stars___3axo0"><div class="gl-star-rating gl-star-rating--small gl-star-rating--black___1T_HV"><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 88%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><div class="gl-star-rating__item"><div class="gl-star-rating__mask" style="width: 12%;"><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__fill" fill="currentColor" stroke="0" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div><svg class="gl-star-rating__star" viewBox="0 0 15 15" xmlns="http://www.w3.org/2000/svg"><path class="gl-star-rating__outline" fill="none" stroke="currentColor" stroke-miterlimit="10" d="M13.277,6.182L9.697,8.782L11.057,12.992L7.487,10.392L3.907,12.992L5.277,8.782L1.697,6.182L6.117,6.182L7.487,1.992L8.857,6.182L13.277,6.182Z"></path></svg></div></div></div><div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">They match practically with  any outfit that I wear!</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div><div class="voting-wrapper___SinHa gl-body gl-body--xs gl-no-margin-bottom gl-vspace"><span>Helpful?</span><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 10.254V9.40572H2V16H6.79143V14.7719M6.79143 10.254C6.79143 10.254 7.94834 9.61701 8.5 9C9.15551 8.26684 9.13417 8.27672 9.66629 6.76801C9.96004 5.93512 10.1454 4.1303 10.1454 4.1303C10.1454 4.1303 11.1213 3.83713 11.5829 4.1303C12.0694 4.43932 12.5411 5.84347 12.5411 6.76801C12.5411 7.69254 12.3375 8.52648 12.3375 8.52648H16.8534C18.77 8.9661 17.8117 12.0434 17.3326 13.8019C16.8534 15.5604 14.9369 15.8697 14.9369 15.8697H9L6.79143 14.7719M6.79143 10.254V14.7719" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div><div class="feedback___3_bp3"><button class="vote-button___1OM9M"><svg width="20" height="20" viewBox="0 0 20 20" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M6.79143 9.74598V10.5943H2V4H6.79143V5.22814M6.79143 9.74598C6.79143 9.74598 7.94834 10.383 8.5 11C9.15551 11.7332 9.13417 11.7233 9.66629 13.232C9.96004 14.0649 10.1454 15.8697 10.1454 15.8697C10.1454 15.8697 11.1213 16.1629 11.5829 15.8697C12.0694 15.5607 12.5411 14.1565 12.5411 13.232C12.5411 12.3075 12.3375 11.4735 12.3375 11.4735H16.8534C18.77 11.0339 17.8117 7.95657 17.3326 6.19809C16.8534 4.43962 14.9369 4.1303 14.9369 4.1303H9L6.79143 5.22814M6.79143 9.74598V5.22814" stroke="black" stroke-linejoin="round"></path></svg></button><span class="feedback-count___2EBZb">0</span></div></div><div class="info-container___kz45Q gl-vspace"><div class="translate-button"></div></div></article>
for review in reviews:
    header12 = review.find_element(By.CSS_SELECTOR, '[class="gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
    #<div class="gl-body gl-no-margin-bottom"><strong class="title___1LLYl clamped___1_0yP">Classic indoor/casual shoes. Best Purchase!</strong></div>
    
    try:
        message12 = review.find_element(By.CSS_SELECTOR, '[class="review-text-container___2NHwE"]').get_attribute("innerHTML")
        #<div class="review-text-container___2NHwE"><div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">great quality! Had one when i was a kid. Glad to to wear it again</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div></div>
    except:
        try:
            message12 = review.find_element(By.CSS_SELECTOR, '[class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom"]').get_attribute("innerHTML")
        except:
            message12 = None
    
    try:
        review_info12 = review.find_element(By.CSS_SELECTOR, '[class="info___PBHlc gl-vspace"]').get_attribute("innerHTML")
        #<div class="info___PBHlc gl-vspace">EeeJR | September 12, 2023 | Verified Purchaser | Incentivized Review</div>
    except:
        review_info12 = None
        
    heading12.append(header12)
    messaging12.append(message12)
    info12.append(review_info12)

Not Found
<span>Adifom Stan Smith Mule Shoes</span>
$70
4 colors available


In [229]:
len(heading12), len(messaging12), len(info12)

(132, 132, 132)

In [230]:
heading12

['<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">So comfy !!!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">N/A</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">It’s Allllll Me*</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Daily wear!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Will recommend this product</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Love them!</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Great value and very comfy</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">Shoes were way to big! Now they are sold out</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">I loveeeee these shoes they are so cute</strong>',
 '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">These clogs have quickly become my favorite go to.</st

In [483]:
heading12_clean = []

for i,head in enumerate(heading12):
    if '<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">' in head:
        clean_head = head.replace('<strong class="title___1LLYl clamped___1_0yP expanded___1YT3r">', "").replace('</strong>',"")
        heading12_clean.append(clean_head)
    else:
        clean_head = head.replace('<strong class="title___1LLYl clamped___1_0yP">', "").replace('</strong>',"")
        heading12_clean.append(clean_head)
heading12_clean

['So comfy !!!',
 'N/A',
 'It’s Allllll Me*',
 'Daily wear!',
 'Will recommend this product',
 'Love them!',
 'Great value and very comfy',
 'Shoes were way to big! Now they are sold out',
 'I loveeeee these shoes they are so cute',
 'These clogs have quickly become my favorite go to.',
 'Very comfy and unique shoes!',
 'Very comfortable …way better than Uggs',
 'Shoes were way to big! Now they are sold out',
 'I loveeeee these shoes they are so cute',
 'These clogs have quickly become my favorite go to.',
 'Very comfy and unique shoes!',
 'Very comfortable …way better than Uggs',
 'These are the perfect stylish throw on shoes.',
 'I love addias',
 'Good',
 'Absolutely amazing price',
 'Best Purchase From Adidas Ever',
 'Super comfy and cute',
 'Great product overall. 10/10 would recommend.',
 'Totally surprised at the comfort level of them',
 'So comfy and with todays style! Love them',
 'Happy with it',
 'Super comfy and cute',
 'Great product overall. 10/10 would recommend.',
 'Tota

In [231]:
messaging12

['<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">I’ve been waiting these pair of shoes so long , finally I got them in Australia , and it’s really comfy .</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">It’s like walking in the clowns\nThe only down thing is you need a size up from your own size as it feels very tight as i end up with a lot of blisters unfortunately and was unable to send them back</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Sooooo Comfy!!!\nSooooo Green!!!\nI’m In Love!!!\nUuu Will Be\nN O T I C E D*</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">I get at least 8 compliments on these every time I wear them.</div>',
 '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">Remember to size down. Will definitely add height on short girls like me.</div>',
 '<div clas

In [484]:
messaging12_clean = []

for i,message in enumerate(messaging12):
    if '<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">' in message:
        clean_message = message.replace('<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom">', "").replace(
     '</div><div class="read-more-btn-container___2Q8_H"><button data-auto-id="review-read-more-button" class="read-more___2dacS gl-link gl-label--m">Read more</button></div>', ""
    )
        messaging12_clean.append(clean_message)
    else:
        clean_message = message.replace('<div class="clamped___1_0yP gl-vspace gl-body gl-no-margin-bottom expanded___1YT3r">', "").replace(
     '</div>', ""
    )
        messaging12_clean.append(clean_message)
    
len(messaging12_clean)

132

In [485]:
messaging12_clean

['I’ve been waiting these pair of shoes so long , finally I got them in Australia , and it’s really comfy .',
 'It’s like walking in the clowns\nThe only down thing is you need a size up from your own size as it feels very tight as i end up with a lot of blisters unfortunately and was unable to send them back',
 'Sooooo Comfy!!!\nSooooo Green!!!\nI’m In Love!!!\nUuu Will Be\nN O T I C E D*',
 'I get at least 8 compliments on these every time I wear them.',
 'Remember to size down. Will definitely add height on short girls like me.',
 'Very cool and colorful!…………………………………………………………………….',
 'it looks chunky and heavy but is actually very light. has a super soft foamy sole but at the same time gives one a full support.',
 'Have to return now because of sizing error and can’t reorder because they are sold out',
 'Best clog ever purchased, they are the super cutest',
 'They are super comfortable and the silver green color is beautiful. I try and wear them with almost everything. Best purchas

In [232]:
info12

['Weiwei | September 14, 2023',
 'Mazz | September 14, 2023 | Verified Purchaser | Incentivized Review',
 'ANT3 | September 14, 2023',
 'Penelope | September 14, 2023 | Verified Purchaser | Incentivized Review',
 'Mjsa01 | September 8, 2023',
 'Kul31 | September 3, 2023',
 'Renren | September 3, 2023 | Verified Purchaser | Incentivized Review',
 'Kita | September 3, 2023',
 'DaDonn | September 1, 2023',
 'Kiwi | August 31, 2023',
 'Neeks4 | August 29, 2023',
 'Twin | August 28, 2023',
 'Kita | September 3, 2023',
 'DaDonn | September 1, 2023',
 'Kiwi | August 31, 2023',
 'Neeks4 | August 29, 2023',
 'Twin | August 28, 2023',
 'ChantMo | August 28, 2023',
 'Ab234!! | August 28, 2023',
 'Nique | August 25, 2023 | Verified Purchaser | Incentivized Review',
 'Rita | August 23, 2023 | Verified Purchaser | Incentivized Review',
 'Donita | August 19, 2023 | Verified Purchaser | Incentivized Review',
 'Nisha215 | August 14, 2023',
 'Perk | August 14, 2023',
 'BeMa | August 13, 2023',
 'Angel21

In [486]:
shoe12 = pd.DataFrame(
    {"Title": [product_name12[6:].replace("</span>", "")]*len(messaging12),
     "Price":[product_price12[1:]]*len(messaging12),
     "ColoursAvailable": [product_colour12[0]]*len(messaging12_clean),
    "ReviewTopic": heading12_clean,
    "Reviews": messaging12_clean,
    "ReviewInfo": info12}
)
shoe12

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,Adifom Stan Smith Mule Shoes,70,4,So comfy !!!,I’ve been waiting these pair of shoes so long ...,"Weiwei | September 14, 2023"
1,Adifom Stan Smith Mule Shoes,70,4,N/A,It’s like walking in the clowns\nThe only down...,"Mazz | September 14, 2023 | Verified Purchaser..."
2,Adifom Stan Smith Mule Shoes,70,4,It’s Allllll Me*,Sooooo Comfy!!!\nSooooo Green!!!\nI’m In Love!...,"ANT3 | September 14, 2023"
3,Adifom Stan Smith Mule Shoes,70,4,Daily wear!,I get at least 8 compliments on these every ti...,"Penelope | September 14, 2023 | Verified Purch..."
4,Adifom Stan Smith Mule Shoes,70,4,Will recommend this product,Remember to size down. Will definitely add hei...,"Mjsa01 | September 8, 2023"
...,...,...,...,...,...,...
127,Adifom Stan Smith Mule Shoes,70,4,I love these they are so comfortable and stylish.,They are good to wear if you are a nurse and w...,None
128,Adifom Stan Smith Mule Shoes,70,4,Very Very Comfortable love the color,Goes well with what I had on had me feeling my...,None
129,Adifom Stan Smith Mule Shoes,70,4,You feel like your walking on air.,"Just love my new adidas gear, never fails me t...",None
130,Adifom Stan Smith Mule Shoes,70,4,I absolutely love them,"Perfect color! Nice, comfortable shoe. I wasn’...",None


In [526]:
df = pd.concat([shoe1, shoe2, shoe3, shoe4, shoe5, shoe6, shoe7, shoe8, shoe9, shoe10, shoe11, shoe12])
df.head()

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,Samba OG Shoes,100,6,Nice quality shoes,They match practically with any outfit that I...,"abdubs35 | September 13, 2023"
1,Samba OG Shoes,100,6,Nice shoes,"Very nice shoes , just not as green as the pic...","THEMAN | September 13, 2023 | Incentivized Review"
2,Samba OG Shoes,100,6,Buy the shoes!,"Bang on trend, comfy and cool. Would recommend...","Huggsy | September 12, 2023 | Verified Purchas..."
3,Samba OG Shoes,100,6,ALL GOODS SOLID! WORTH THE PRICE!,THAT WAS DOPE! medyo mahaba lng ng very little...,"TOYOTABOY | September 12, 2023"
4,Samba OG Shoes,100,6,Never out of style,Love how light on your feet they are and comfo...,"SangeBo | September 12, 2023"


In [527]:
df.head(10)

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,ReviewInfo
0,Samba OG Shoes,100,6,Nice quality shoes,They match practically with any outfit that I...,"abdubs35 | September 13, 2023"
1,Samba OG Shoes,100,6,Nice shoes,"Very nice shoes , just not as green as the pic...","THEMAN | September 13, 2023 | Incentivized Review"
2,Samba OG Shoes,100,6,Buy the shoes!,"Bang on trend, comfy and cool. Would recommend...","Huggsy | September 12, 2023 | Verified Purchas..."
3,Samba OG Shoes,100,6,ALL GOODS SOLID! WORTH THE PRICE!,THAT WAS DOPE! medyo mahaba lng ng very little...,"TOYOTABOY | September 12, 2023"
4,Samba OG Shoes,100,6,Never out of style,Love how light on your feet they are and comfo...,"SangeBo | September 12, 2023"
5,Samba OG Shoes,100,6,The perfect everyday shoe,You can wear these shoes with anything and the...,"Herryhernadez | September 12, 2023"
6,Samba OG Shoes,100,6,Super nice shoes,"Love the colorway and the way it fits, what an...","Vides | September 12, 2023"
7,Samba OG Shoes,100,6,ALL GOODS SOLID! WORTH THE PRICE!,THAT WAS DOPE! medyo mahaba lng ng very little...,"TOYOTABOY | September 12, 2023"
8,Samba OG Shoes,100,6,Never out of style,Love how light on your feet they are and comfo...,"SangeBo | September 12, 2023"
9,Samba OG Shoes,100,6,The perfect everyday shoe,You can wear these shoes with anything and the...,"Herryhernadez | September 12, 2023"


In [528]:
df.to_csv("adidas_extracted_shoe_data.csv")

In [491]:
data = df.drop_duplicates()

In [514]:
data.iloc[1001]["ReviewInfo"].split("|")

['JennyP ', ' July 15, 2023 ', ' Verified Purchaser ', ' Incentivized Review']

In [518]:
users = []
dates = []
verifys = []
reviews= []

for dat in data["ReviewInfo"]:
    
    if dat is not None:
        dat_split = dat.split("|")
        if len(dat_split) >= 4:
            users.append(dat_split[0])
            dates.append(dat_split[1])
            verifys.append(True)
            reviews.append(True)
        elif len(dat_split) == 3:
            users.append(dat_split[0])
            dates.append(dat_split[1])
            if dat_split[2] == ' Verified Purchaser ':
                verifys.append(True)
                reviews.append(False)
            else:
                verifys.append(False)
                reviews.append(True)
        else:
            users.append(dat_split[0])
            dates.append(dat_split[1])
            verifys.append(False)
            reviews.append(False)
            
    else:
        users.append(None)
        dates.append(None)
        verifys.append(False)
        reviews.append(False)
        
print(len(users), len(dates), len(verifys), len(reviews))

1106 1106 1106 1106


In [520]:
assert len(users) == len(data)

In [522]:
data["UserID"] = users 
data["Date"] = dates
data["VerifiedPurchaser"] = verifys
data["IncentivizedReview"] = reviews

In [524]:
data.drop(columns="ReviewInfo", inplace=True)
data.head(30)

,Title,Price,ColoursAvailable,ReviewTopic,Reviews,UserID,Date,VerifiedPurchaser,IncentivizedReview
0,Samba OG Shoes,100,6,Nice quality shoes,They match practically with any outfit that I...,abdubs35,"September 13, 2023",False,False
1,Samba OG Shoes,100,6,Nice shoes,"Very nice shoes , just not as green as the pic...",THEMAN,"September 13, 2023",False,True
2,Samba OG Shoes,100,6,Buy the shoes!,"Bang on trend, comfy and cool. Would recommend...",Huggsy,"September 12, 2023",True,True
3,Samba OG Shoes,100,6,ALL GOODS SOLID! WORTH THE PRICE!,THAT WAS DOPE! medyo mahaba lng ng very little...,TOYOTABOY,"September 12, 2023",False,False
4,Samba OG Shoes,100,6,Never out of style,Love how light on your feet they are and comfo...,SangeBo,"September 12, 2023",False,False
5,Samba OG Shoes,100,6,The perfect everyday shoe,You can wear these shoes with anything and the...,Herryhernadez,"September 12, 2023",False,False
6,Samba OG Shoes,100,6,Super nice shoes,"Love the colorway and the way it fits, what an...",Vides,"September 12, 2023",False,False
11,Samba OG Shoes,100,6,"Nice kicks,size up!","Love the trainers, but had to return as fit is...",Ali7,"September 12, 2023",True,True
20,Samba OG Shoes,100,6,"I wore these in Italy for 16,000 steps/day. Gr...",I love these shoes so much they can be dressed...,Julie0515,"September 12, 2023",False,False
21,Samba OG Shoes,100,6,So so,The back is too high and rubs when walking i s...,Bear,"September 12, 2023",True,True


In [525]:
data.to_csv("adidas_data_15-09-2023.csv")